In [1]:
# Dependencies.
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
# Open text file with HTML containing links to each location.
with open('../resources/pei_gov_placefinder_links.txt') as f:
    data = f.read()

# Split the read data into lines to loop through.
data_lines = data.split('<a ')

In [3]:
# Store the name and corresponding url.
name_url = []

# Loop through each line.
for line in data_lines:
    # Locate the URL string.
    u = re.search(r'"(.+?")', line)
    if u:
        # Save the URL string and locate and save the name string.
        url = u.group(1)[:-1].replace('amp;', '')
        name = re.search(r'>(.+?<)', line).group(1)[:-1]
        name_url.append([name, url])

In [4]:
# Store the combinations.
name_url_loc = []

# Loop through the names and store the coordinates from each webpage.
for place in name_url:

    # Create location lookup URL.
    lu_url = 'http://www.gov.pe.ca/placefinder/' + place[1]
    
    # Request the page.
    page = requests.get(lu_url)
    
    # Create the soup.
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Get the string of text containing the location.
    location_string = soup.find_all(class_='copy')[0].text

    # Parse text and get the coordinates.
    loc_re = re.search(r"N (.+?\.)", location_string)
    coords = loc_re.group(1)[:-1]
    
    # Store the three variables.
    name_url_loc.append([place[0], place[1], coords])

In [5]:
# Create a DataFrame.
coords_df = pd.DataFrame(name_url_loc, columns=['Place', 'URL', 'Coords'])
coords_df = coords_df.set_index('Place')
coords_df.to_csv('../resources/transformed_data/pei_places_coordinates.csv')
coords_df.head()

,URL,Coords
Place,,
48 Road,index.php3?cgndb=BAAFX&formattype=&city=48%20Road,"46°15', W 62°43'"
Abney,index.php3?cgndb=BADOL&formattype=&city=Abney,"45°59', W 62°34'"
Abrams Village,index.php3?cgndb=BADOM&formattype=&city=Abrams...,"46°27', W 64°05'"
Afton,index.php3?cgndb=BAEDL&formattype=&city=Afton,"46°11', W 63°11'"
Afton Road,index.php3?cgndb=BADOP&formattype=&city=Afton%...,"46°23', W 62°56'"
